In [20]:
import requests

: 

In [1]:
# import requests
# import json

# # Define the Ollama server URL
# ollama_url = "http://127.0.0.1:11434/api/generate"

# # Test connection to Ollama
# def test_connection():
#     # Simple payload with a basic prompt
#     payload = {
#         "model": "llama2",
#         "prompt": "Hello, how are you?"
#     }
    
#     try:
#         response = requests.post(ollama_url, json=payload, stream=True)
        
#         # Process the response line by line
#         print("Connection successful! Streamed response:")
#         for line in response.iter_lines():
#             if line:
#                 try:
#                     # Parse each line as a separate JSON object
#                     data = json.loads(line.decode('utf-8'))
#                     print("Partial response:", data, data.get("response"))
#                 except json.JSONDecodeError as e:
#                     print("JSON decoding error:", e)
                    
#     except Exception as e:
#         print("Error connecting to Ollama:", str(e))

# # Run the connection test
# test_connection()


In [2]:
# import requests
# import json

# # Define the Ollama server URL
# ollama_url = "http://127.0.0.1:11434/api/generate"

# # Test connection to Ollama
# def test_connection():
#     # Simple payload with a basic prompt
#     payload = {
#         "model": "llama2",
#         "prompt": "Hello, how are you?"
#     }
    
#     try:
#         response = requests.post(ollama_url, json=payload, stream=True)
        
#         # Process the response line by line
#         print("Connection successful! Streamed response:")
#         for line in response.iter_lines():
#             if line:
#                 try:
#                     # Parse each line as a separate JSON object
#                     data = json.loads(line.decode('utf-8'))
#                     print("Partial response:", data.get("response"))
#                 except json.JSONDecodeError as e:
#                     print("JSON decoding error:", e)
                    
#     except Exception as e:
#         print("Error connecting to Ollama:", str(e))

# # Run the connection test
# test_connection()


In [3]:
import requests
import json

# Define the Ollama server URL
ollama_url = "http://127.0.0.1:11434/api/generate"

# Test connection to Ollama and gather the response in one line
def test_connection():
    # Simple payload with a basic prompt
    payload = {
        "model": "llama2",
        "prompt": "Can you help me build summarize a text and further go on and find it's sentiment?"
    }
    
    try:
        response = requests.post(ollama_url, json=payload, stream=True)
        
        # Collect each partial response into a list
        full_response = []
        
        for line in response.iter_lines():
            if line:
                try:
                    # Parse each line as a separate JSON object
                    data = json.loads(line.decode('utf-8'))
                    # Append the "response" field to the list
                    partial_response = data.get("response", "")
                    full_response.append(partial_response)
                except json.JSONDecodeError as e:
                    print("JSON decoding error:", e)
        
        # Join all parts into a single string
        final_response = "".join(full_response)
        print("Final response:", final_response)
        
    except Exception as e:
        print("Error connecting to Ollama:", str(e))

# Run the connection test
test_connection()


Final response: Of course! I'd be happy to help you summarize and analyze the sentiment of a text. Can you please provide the text you'd like me to summarize and analyze?


In [4]:
import requests
import json
import re

ollama_url = "http://127.0.0.1:11434/api/generate"

def summarize_text(text):
    payload = {
        "model": "llama2",
        "prompt": f"Summarize the following text:\n\n{text}\n\nSummary:"
    }
    
    response = requests.post(ollama_url, json=payload, stream=True)
    full_response = []
    
    for line in response.iter_lines():
        if line:
            data = json.loads(line.decode('utf-8'))
            partial_response = data.get("response", "")
            full_response.append(partial_response)
    # print('Summary ',"".join(full_response))
    return "".join(full_response)

def analyze_sentiment(text):
    payload = {
        "model": "llama2",
        "prompt": f"Analyze the sentiment of the following text and provide just the sentiment scores in the format {{'neg': , 'neu': , 'pos': , 'compound': }}:\n\n{text}\n\nSentiment Scores: in 2 decimals . Please do not output any other text in your response"
    }
    
    response = requests.post(ollama_url, json=payload, stream=True)
    full_response = []
    
    for line in response.iter_lines():
        if line:
            data = json.loads(line.decode('utf-8'))
            partial_response = data.get("response", "")
            full_response.append(partial_response)
    
    response_text = "".join(full_response)

    scores = {
        "neg": 0.0,
        "neu": 0.0,
        "pos": 0.0,
        "compound": 0.0
    }
    
    match = re.search(r"\{(.*)\}", response_text)
    # scores['neg'] = match.group(1).split(',')[0].split(': ')[1]
    # sent_scores = [match.group(1).split(',')[i].split(': ')[1] for i in [0,1,2,3]]
    # print(match.group(1))
    if len(match.group(1).split(',')) > 0:
        scores['neg'] = match.group(1).split(',')[0].split(': ')[1]
    if len(match.group(1).split(',')) > 1:
        scores['neu'] = match.group(1).split(',')[1].split(': ')[1]
    if len(match.group(1).split(',')) > 2:
        scores['pos'] = match.group(1).split(',')[2].split(': ')[1]
    if len(match.group(1).split(',')) > 3:
        scores['compound'] = match.group(1).split(',')[3].split(': ')[1]
    # print(scores)
    
    return scores

# text = """
# OpenAI's GPT-4 model has significantly improved the capabilities of language models.
# Users find it more accurate and versatile, making it suitable for both casual and professional tasks.
# However, it still has limitations and can sometimes generate incorrect responses.
# """

# summary = summarize_text(text)
# print("Summary:", summary)

# sentiment = analyze_sentiment(summary)
# print("Sentiment:", sentiment)

In [5]:
import pandas as pd
import ast

news_data = pd.read_csv('Apple_News_Articles_2013_1_2024_10.csv')
filtered_columns_data = news_data[['abstract','snippet','lead_paragraph','headline','pub_date']]

filtered_columns_data = pd.concat([filtered_columns_data.drop('headline',axis = 1),pd.json_normalize([ast.literal_eval(item) for item in list(filtered_columns_data['headline'])])[['main','kicker','print_headline']]],axis = 1)
print('Checking for null values')
display({i:filtered_columns_data[i].isna().sum() for i in filtered_columns_data})
print('Removing NULL values')
display([filtered_columns_data[i].fillna('',inplace=True) for i in filtered_columns_data])
print('Checking for null values')
display({i:filtered_columns_data[i].isna().sum() for i in filtered_columns_data})

text = []
for j in range(filtered_columns_data.shape[0]):
    str1=''
    for i in [0,1,2,4,5,6]:
        str1 += filtered_columns_data.iloc[j,i]
    text.append(str1)
# filtered_columns_data.iloc[:,5:]
filtered_columns_data['text_in_article'] = text
display(filtered_columns_data.columns)
display(filtered_columns_data.shape)


Checking for null values


{'abstract': 3,
 'snippet': 20,
 'lead_paragraph': 14,
 'pub_date': 0,
 'main': 0,
 'kicker': 1569,
 'print_headline': 0}

Removing NULL values


C:\Users\janan\AppData\Local\Temp\ipykernel_19788\1103216963.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  display([filtered_columns_data[i].fillna('',inplace=True) for i in filtered_columns_data])


[None, None, None, None, None, None, None]

Checking for null values


{'abstract': 0,
 'snippet': 0,
 'lead_paragraph': 0,
 'pub_date': 0,
 'main': 0,
 'kicker': 0,
 'print_headline': 0}

Index(['abstract', 'snippet', 'lead_paragraph', 'pub_date', 'main', 'kicker',
       'print_headline', 'text_in_article'],
      dtype='object')

(2712, 8)

In [ ]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

df = filtered_columns_data

with ThreadPoolExecutor() as executor:
    df['summary'] = list(tqdm(executor.map(summarize_text, df['text_in_article']), total=len(df), desc="Summarizing"))



Summarizing:   0%|▎                                                                            | 11/2712 [11:50<32:12:12, 42.92s/it]

In [ ]:
# pip install tqdm

import pandas as pd
from tqdm import tqdm

tqdm.pandas()  # Register `tqdm` with pandas


In [ ]:
df['sentiment'] = df['summary'].progress_apply(analyze_sentiment)
df_sentiment = df['sentiment'].apply(pd.Series)
df = pd.concat([df, df_sentiment], axis=1)
df.head(5)

  0%|                                                                                           | 2/2712 [01:11<27:01:15, 35.90s/it]

'neg': 0.6, 'neu': 0.4, 'pos': 0.0, 'compound': 0.0


  0%|                                                                                           | 3/2712 [01:33<22:42:10, 30.17s/it]

'neg': 0.16, 'neu': 0.57, 'pos': 0.27, 'compound': 0.0


  0%|▏                                                                                          | 4/2712 [01:51<19:06:34, 25.40s/it]

...

'neg': 0.15, 'neu': 0.78, 'pos': 0.94, 'compound': 0.89


 16%|██████████████▏                                                                        | 442/2712 [4:11:59<27:44:33, 44.00s/it]

'neg': 0.3, 'neu': 0.4, 'pos': 0.3, 'compound': 0.1


 16%|██████████████▏                                                                        | 442/2712 [4:28:14<22:57:36, 36.41s/it]


KeyboardInterrupt: 